Skew isn't working!

<a id="toc"></a>

# <u>Table of Contents</u>
1.) [TODO](#todo)  
2.) [Imports](#imports)  
3.) [Load data](#load)  
4.) [Bureau Balance](#bureau_bal)  
&nbsp;&nbsp;&nbsp;&nbsp; 4.1.) [Merge into Bureau](#merge_bureau_bal)  
5.) [POS CASH balance](#pos_cash)  
&nbsp;&nbsp;&nbsp;&nbsp; 5.1.) [Missing values](#pos_nan)  
&nbsp;&nbsp;&nbsp;&nbsp; 5.2.) [Merge into Previous Application](#merge_pos_cash)  
6.) [Installment Payments](#install_pay)  
&nbsp;&nbsp;&nbsp;&nbsp; 6.1.) [Missing values](#install_nan)  
&nbsp;&nbsp;&nbsp;&nbsp; 6.2.) [Merge into Previous Application](#merge_install_pay)  
7.) [Credit Card Balance](#credit)  
&nbsp;&nbsp;&nbsp;&nbsp; 7.1.) [Missing values](#credit_nan)  
&nbsp;&nbsp;&nbsp;&nbsp; 7.2.) [Merge into Previous Application](#merge_credit)  
8.) [Final Data Prep](#final_merge)  
&nbsp;&nbsp;&nbsp;&nbsp; 8.1.) [Missing values](#final_nan)  
9.) [Modeling](#models)  
10.) [Save file to CSV](#save)  

<a id="todo"></a>

# [^](#toc) <u>TODO</u>

- Fix skew on columns (prev_app -> AMT_ANNUITY + AMT_APPLICATION)
- Tinker with the best way to replace missing values (dropping cols?)
- Look for outliers
- Remove large outlier values from distributions
- Include timeline relatoinships like MONTHS_BALANCE
- Tune model parameters
- Address [this](https://www.kaggle.com/c/home-credit-default-risk/discussion/57248)

---
<a id="imports"></a>

# [^](#toc) <u>Imports</u>

In [2]:
### Standard imports
import pandas as pd
import numpy as np

# Time keeper
import time

# Statistics importrs
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p

# Progress bar
from tqdm import tqdm

### Removes warnings from output
import warnings
warnings.filterwarnings('ignore')

### Helper functions

In [3]:
# function to create dummy variables of categorical features
def get_dummies(df, cats):
    for col in cats:
        df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df 

def fillna_num(df):
    missing_cols = [col for col in df.columns if any(df[col].isnull()) and df[col].dtype != object]
    for col in missing_cols:
        df[col] = df[col].fillna(df[col].median())
    return df

def fillna_cat(df):
    for col in [col for col in df if df[col].dtype==object]:
        df[col] = df[col].fillna(df[col].mode()[0])
    return df

def factorize_df(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 

---
<a id="load"></a>

# [^](#toc) <u>Load data</u>

In [4]:
DATA_PATH = "../data/home_default/"

bureau   = pd.read_csv(DATA_PATH + "bureau.csv")
prev_app = pd.read_csv(DATA_PATH + "previous_application.csv")

print("Shape of bureau:",    bureau.shape)
print("Shape of prev_app:",  prev_app.shape)

Shape of bureau: (1716428, 17)
Shape of prev_app: (1670214, 37)


### Missing Values - Bureau

In [5]:
cols = ["DAYS_CREDIT_ENDDATE", "DAYS_ENDDATE_FACT", "AMT_CREDIT_MAX_OVERDUE",
        "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT",
        "AMT_ANNUITY"]

for col in tqdm(cols):
    bureau[col + "_nan"] = bureau[col].map(lambda x: 1 if np.isnan(x) else 0)
#     mode                 = bureau[bureau[col].notnull()][col].mode().iloc[0]
    bureau[col]          = bureau[col].fillna(0)
    
sum(bureau.isnull().sum())

100%|██████████| 7/7 [00:15<00:00,  2.21s/it]


0

### Missing values - Previous application

In [6]:
### Fillin missing values with filler... NaN
for col in ("NAME_TYPE_SUITE", "PRODUCT_COMBINATION"):
    prev_app[col] = prev_app[col].fillna("NaN")
    
### These values have to little counts, clump as one
misc_vals = ["Hobby", "Money for a third person", "Refusal to name the goal"]
misc_map = lambda x: "Misc" if x in misc_vals else x
prev_app.NAME_CASH_LOAN_PURPOSE = prev_app.NAME_CASH_LOAN_PURPOSE.map(misc_map)

### Drop outliers
prev_app = prev_app.drop(
                            prev_app[(prev_app.NAME_GOODS_CATEGORY == "Animals") |
                                     (prev_app.NAME_GOODS_CATEGORY == "House Construction")].index)

### Light missing values
prev_app["AMT_CREDIT"] = prev_app["AMT_CREDIT"].fillna(prev_app["AMT_CREDIT"].median())

### Moderate missing values
for col in ("AMT_ANNUITY", "AMT_GOODS_PRICE", "CNT_PAYMENT", "AMT_DOWN_PAYMENT", "RATE_DOWN_PAYMENT",
            "DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION", "DAYS_LAST_DUE",
            "DAYS_TERMINATION", "NFLAG_INSURED_ON_APPROVAL"):
    prev_app[col + "_nan"] = prev_app[col].map(lambda x: 1 if np.isnan(x) else 0)
    prev_app[col] = prev_app[col].transform(lambda x: x.fillna(x.median()))
    
### Severe missing values
for col in ("RATE_INTEREST_PRIMARY", "RATE_INTEREST_PRIVILEGED"):
    prev_app[col] = prev_app[col].map(lambda x: 0 if np.isnan(x) else 1)
    
sum(prev_app.isnull().sum())

0

<a id="bureau_bal"></a>

# [^](#toc) <u>Bureau Balance</u>

In [7]:
bureau_balance = pd.read_csv(DATA_PATH + "bureau_balance.csv")
print("Shape of bureau_balance:",  bureau_balance.shape)

print("\nColumns of bureau_balance:")
print(" --- ".join(bureau_balance.columns.values))

Shape of bureau_balance: (27299925, 3)

Columns of bureau_balance:
SK_ID_BUREAU --- MONTHS_BALANCE --- STATUS


### Setup bureau balance - get dummies

In [8]:
merge_df = get_dummies(bureau_balance, ["STATUS"])

<a id="merge_bureau_bal"></a>

### [^](#toc) <u>Merge into Bureau</u>

FIXME: select features (1) max months, (2) most recent status (or few)), and (3) time weighted status

In [9]:
merge_df = merge_df.drop(["MONTHS_BALANCE", "STATUS"], axis=1)

# prep for merge
merge_df = merge_df.groupby("SK_ID_BUREAU").sum().reset_index()

### Add the median of the rest of the columns
right    = bureau_balance.groupby("SK_ID_BUREAU").median().reset_index()
merge_df = merge_df.merge(right=right, how="left", on="SK_ID_BUREAU").set_index("SK_ID_BUREAU")

### Remember added columns
merged_cols = ['bur_bal_' + col for col in merge_df.columns]
merge_df.columns = merged_cols

# Merge
bureau = bureau.merge(right=merge_df.reset_index(), how='left', on='SK_ID_BUREAU')

### Fill in new missing values

In [10]:
bureau["no_bureau_bal"] = bureau[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)
bureau[merged_cols]     = bureau[merged_cols].fillna(0)

---
<a id="pos_cash"></a>

# [^](#toc) <u>POS CASH balance</u>

In [11]:
pcb = pd.read_csv(DATA_PATH + "POS_CASH_balance.csv")
print("Shape of pcb:",  pcb.shape)

print("\nColumns of pcb:")
print(" --- ".join(pcb.columns.values))

Shape of pcb: (10001358, 8)

Columns of pcb:
SK_ID_PREV --- SK_ID_CURR --- MONTHS_BALANCE --- CNT_INSTALMENT --- CNT_INSTALMENT_FUTURE --- NAME_CONTRACT_STATUS --- SK_DPD --- SK_DPD_DEF


<a id="pos_nan"></a>

### [^](#toc) Missing Values

In [12]:
for col in ("CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"):
    pcb[col] = pcb[col].transform(lambda x: x.fillna(x.median()))

### Remove Outliers

In [13]:
pcb = pcb.drop(pcb[pcb.NAME_CONTRACT_STATUS.isin(["XNA", "Canceled"])].index)

### Get Dummies

In [14]:
merge_df = pcb[["SK_ID_PREV", "NAME_CONTRACT_STATUS"]]

merge_df = get_dummies(merge_df, ["NAME_CONTRACT_STATUS"])
merge_df = merge_df.drop("NAME_CONTRACT_STATUS", axis=1)

<a id="merge_pos_cash"></a>

### [^](#toc) <u>Merge into Previous Application</u>

In [15]:
# prep for merge
count    = merge_df.groupby("SK_ID_PREV").count()
merge_df = merge_df.groupby("SK_ID_PREV").sum().reset_index()
merge_df["N"] = list(count.iloc[:,0])

right    = pcb.drop("SK_ID_CURR", axis=1).groupby("SK_ID_PREV").median().reset_index()
merge_df = merge_df.merge(right=right, how="left", on="SK_ID_PREV").set_index("SK_ID_PREV")

merged_cols = ['pos_' + col for col in merge_df.columns]
merge_df.columns = merged_cols

# Merge
prev_app = prev_app.merge(right=merge_df.reset_index(), how='left', on='SK_ID_PREV')

### Fill in missing values

In [16]:
prev_app["no_pcb"] = prev_app[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null      = prev_app[col].notnull()
    mode          = prev_app[not_null][col].mode()
    prev_app[col] = prev_app[col].fillna(mode)    

100%|██████████| 13/13 [00:10<00:00,  1.29it/s]


---
<a id="install_pay"></a>

# [^](#toc) <u>Installment Payments</u>

In [17]:
install_pay = pd.read_csv(DATA_PATH + "installments_payments.csv")
print("Shape of install_pay:",  install_pay.shape)

print("\nColumns of install_pay:")
print(" --- ".join(install_pay.columns.values))

Shape of install_pay: (13605401, 8)

Columns of install_pay:
SK_ID_PREV --- SK_ID_CURR --- NUM_INSTALMENT_VERSION --- NUM_INSTALMENT_NUMBER --- DAYS_INSTALMENT --- DAYS_ENTRY_PAYMENT --- AMT_INSTALMENT --- AMT_PAYMENT


<a id="install_nan"></a>

### [^](#toc) <u>Missing values</u>

In [18]:
for col in ("DAYS_ENTRY_PAYMENT", "AMT_PAYMENT"):
    install_pay[col + "_nan"] = install_pay[col].map(lambda x: 1 if np.isnan(x) else 0)
    install_pay[col] = install_pay[col].fillna(0)

### Setup for merge

In [19]:
install_pay["AMT_MISSING"] = install_pay["AMT_INSTALMENT"] - install_pay["AMT_PAYMENT"]
temp = install_pay.groupby("SK_ID_PREV")["AMT_MISSING"]

merge_df = pd.DataFrame({
    "INSTALL_missing_max": temp.max(),
    "INSTALL_missing_min": temp.min(),
    "INSTALL_missing_med": temp.median(),
    "INSTALL_payment_nan": install_pay.groupby("SK_ID_PREV")["AMT_PAYMENT_nan"].sum(),
    "INSTALL_N":           temp.count()
})

### Add the rest of the columns

In [20]:
right = install_pay.drop("SK_ID_CURR", axis=1).groupby("SK_ID_PREV").median().reset_index()
merge_df = merge_df.reset_index()

merge_df = merge_df.merge(right=right, how="left", on="SK_ID_PREV").set_index("SK_ID_PREV")
merged_cols = merge_df.columns

<a id="merge_install_pay"></a>

### [^](#toc) <u>Merge into Previous Application</u>

In [21]:
# Merge
prev_app = prev_app.merge(right=merge_df.reset_index(), how='left', on='SK_ID_PREV')

### Fill in missing values

In [22]:
prev_app["no_install"] = prev_app[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null      = prev_app[col].notnull()
    mode          = prev_app[not_null][col].mode()
    prev_app[col] = prev_app[col].fillna(mode)    

100%|██████████| 14/14 [00:15<00:00,  1.09s/it]


---
<a id="credit"></a>

# [^](#toc) <u>Credit Card Balance</u>

FIXME: select (1) max months of credit history

In [23]:
credit_card = pd.read_csv(DATA_PATH + "credit_card_balance.csv")
print("Shape of credit_card:",  credit_card.shape)

print("\nColumns of credit_card:")
print(" --- ".join(credit_card.columns.values))

Shape of credit_card: (3840312, 23)

Columns of credit_card:
SK_ID_PREV --- SK_ID_CURR --- MONTHS_BALANCE --- AMT_BALANCE --- AMT_CREDIT_LIMIT_ACTUAL --- AMT_DRAWINGS_ATM_CURRENT --- AMT_DRAWINGS_CURRENT --- AMT_DRAWINGS_OTHER_CURRENT --- AMT_DRAWINGS_POS_CURRENT --- AMT_INST_MIN_REGULARITY --- AMT_PAYMENT_CURRENT --- AMT_PAYMENT_TOTAL_CURRENT --- AMT_RECEIVABLE_PRINCIPAL --- AMT_RECIVABLE --- AMT_TOTAL_RECEIVABLE --- CNT_DRAWINGS_ATM_CURRENT --- CNT_DRAWINGS_CURRENT --- CNT_DRAWINGS_OTHER_CURRENT --- CNT_DRAWINGS_POS_CURRENT --- CNT_INSTALMENT_MATURE_CUM --- NAME_CONTRACT_STATUS --- SK_DPD --- SK_DPD_DEF


<a id="credit_nan"></a>

### [^](#toc) <u>Missing Values and Outliers</u>

In [24]:
# ------------------------------
### Remove outliers
# Gets indices with outlier values
temp = credit_card[credit_card.NAME_CONTRACT_STATUS.isin(["Refused", "Approved"])].index

# Drops outlier values
credit_card = credit_card.drop(temp, axis=0)

# ------------------------------
#### Fill in missing values
cols = [
        "AMT_DRAWINGS_ATM_CURRENT", "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_POS_CURRENT", 
        "AMT_INST_MIN_REGULARITY", "AMT_PAYMENT_CURRENT", "CNT_DRAWINGS_ATM_CURRENT", 
        "CNT_DRAWINGS_OTHER_CURRENT", "CNT_DRAWINGS_POS_CURRENT", "CNT_INSTALMENT_MATURE_CUM"
]
for col in tqdm(cols):
    not_null = credit_card[col].notnull()
    mode = float(credit_card[not_null][col].mode())
    credit_card[col] = credit_card[col].fillna(mode)

100%|██████████| 9/9 [00:10<00:00,  1.11s/it]


### Setup Categorical column

In [25]:
temp = credit_card[["SK_ID_PREV", "NAME_CONTRACT_STATUS"]]

temp = get_dummies(temp, ["NAME_CONTRACT_STATUS"])
temp = temp.drop("NAME_CONTRACT_STATUS", axis=1)
temp = temp.groupby("SK_ID_PREV").sum()

### Select columns

In [26]:
merge_df = pd.DataFrame({
    "AMT_BALANCE": credit_card.groupby("SK_ID_PREV").AMT_BALANCE.mean(),
    "SK_DPD":      credit_card.groupby("SK_ID_PREV").SK_DPD.max(),
    "SK_DPD_DEF":  credit_card.groupby("SK_ID_PREV").SK_DPD_DEF.max(),
    "N":           credit_card.groupby("SK_ID_PREV").count().iloc[:,0]
})

merge_df = temp.join(merge_df)
del temp

### Add the rest of the columns

In [27]:
right = credit_card.drop("SK_ID_CURR", axis=1).groupby("SK_ID_PREV").median().reset_index()
merge_df = merge_df.reset_index()
merge_df = merge_df.merge(right=right, how="left", on="SK_ID_PREV").set_index("SK_ID_PREV")

<a id="merge_credit"></a>

### [^](#toc) <u>Merge into Previous Application</u>

In [28]:
# Merge
merged_cols = ['credit_' + col for col in merge_df.columns]
merge_df.columns = merged_cols
prev_app = prev_app.merge(right=merge_df.reset_index(), how='left', on='SK_ID_PREV')

### Fill in new NaN values

In [29]:
prev_app["no_credit"] = prev_app[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null = prev_app[col].notnull()
    median = prev_app[not_null][col].median()
    prev_app[col] = prev_app[col].fillna(median)    

100%|██████████| 29/29 [00:08<00:00,  3.37it/s]


---

### Misc clean up

In [30]:
### Drop unneeded SK_ID_PREV from prev_app
prev_app = prev_app.drop("SK_ID_PREV", axis=1)

print("Number of null in prev_app:", sum(prev_app.isnull().sum()))
print("Number of null in bureau:  ", sum(bureau.isnull().sum()))

Number of null in prev_app: 19985355
Number of null in bureau:   0


---
<a id="final_merge"></a>

# [^](#toc) <u>Final Data Prep</u>

In [59]:
train = pd.read_csv(DATA_PATH + "train.csv")
test  = pd.read_csv(DATA_PATH + "test.csv")

print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Shape of train: (307511, 122)
Shape of test: (48744, 121)


### Split into predictors, target, and id

In [60]:
train_y = train.TARGET
train_x = train.drop(["TARGET"], axis=1)

test_id = test.SK_ID_CURR
test_x  = test

### Merge train and test data

In [61]:
full    = pd.concat([train_x, test_x])
train_N = len(train_x)

<a id="final_nan"></a>

### [^](#toc) <u>Missing values</u>

In [62]:
full = fillna_cat(full)
full = fillna_num(full)
sum(full.isnull().sum())

0

### Get dummies

In [63]:
prev_app = pd.get_dummies(prev_app)
bureau   = pd.get_dummies(bureau)

### Factorize

In [64]:
# Get categorical features
data_cats = [col for col in full.columns if full[col].dtype == 'object']

# Factorize the dataframe
full = factorize_df(full, data_cats)

### Merge Previous Application with full

In [65]:
merge_df      = prev_app.groupby('SK_ID_CURR').mean()
merge_df["N"] = prev_app.groupby('SK_ID_CURR').count().iloc[:,0]
merged_cols   = ['p_' + col for col in merge_df.columns]
merge_df.columns = merged_cols

full = full.merge(right=merge_df.reset_index(), how='left', on='SK_ID_CURR')

#### Fill NaN values

In [66]:
full["no_prev_app"] = full[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null  = full[col].notnull()
    median    = full[not_null][col].median()
    full[col] = full[col].fillna(median)    
    
sum(full.isnull().sum())

100%|██████████| 231/231 [03:40<00:00,  1.05it/s]


0

### Merge Bureau with full

In [67]:
# Average Values for all bureau features 
merge_df         = bureau.groupby('SK_ID_CURR').mean()
merge_df['N']    = bureau.groupby('SK_ID_CURR').count().iloc[:,0]
merged_cols = ['b_' + f_ for f_ in merge_df.columns]
merge_df.columns = merged_cols

full = full.merge(right=merge_df.reset_index(), how='left', on='SK_ID_CURR')

#### Fill NaN values

In [68]:
full["no_bureau"] = full[merged_cols[0]].map(lambda x: 1 if np.isnan(x) else 0)

for col in tqdm(merged_cols):
    not_null  = full[col].notnull()
    median    = full[not_null][col].median()
    full[col] = full[col].fillna(median)    

sum(full.isnull().sum())

100%|██████████| 54/54 [00:49<00:00,  1.08it/s]


0

### Drop SK_ID_CURR

In [69]:
full = full.drop("SK_ID_CURR", axis=1)

### Handle skew

In [70]:
num_cols = [col for col in full.columns if (full[col].dtype != object and "nan" not in col)]

# Check the skew of all numerical features
skewed_feats = full[num_cols].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

skewness = pd.DataFrame({'Skew' :skewed_feats})
skewed_features = skewness[abs(skewness) > 0.75].index

lam = 0.15

for feat in tqdm(skewed_features):
#     full[feat].dtype == int
    temp = full[feat].unique()
    if len(temp) > 4:
        full[feat] = boxcox1p(full[feat], lam)

100%|██████████| 386/386 [00:07<00:00, 54.50it/s]


### Split full back into train and test

In [71]:
train_x = full[:train_N]
test_x = full[train_N:]

### Processed data look

In [72]:
train_x.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,b_CREDIT_TYPE_Loan for purchase of shares (margin lending),b_CREDIT_TYPE_Loan for the purchase of equipment,b_CREDIT_TYPE_Loan for working capital replenishment,b_CREDIT_TYPE_Microloan,b_CREDIT_TYPE_Mobile operator loan,b_CREDIT_TYPE_Mortgage,b_CREDIT_TYPE_Real estate loan,b_CREDIT_TYPE_Unknown type of loan,b_N,no_bureau
0,0,0,0,0,0.0,35.008065,39.601640,23.729390,38.592338,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.602594,0
1,0,1,0,1,0.0,36.845784,48.372758,25.455739,47.264738,0.730463,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.820334,0
2,1,0,1,0,0.0,28.676485,32.548972,18.354869,32.548972,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.194318,0
3,0,1,0,0,0.0,32.548972,37.814328,24.579359,37.472327,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.820334,1
4,0,0,0,0,0.0,31.934081,41.243367,23.178612,41.243367,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.730463,0


<a id="models"></a>

# [^](#toc) <u>Models </u>

### sban's method

In [76]:
from sklearn.model_selection import train_test_split 
import lightgbm as lgb

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=17)
lgb_train = lgb.Dataset(data=train_x, label=train_y)
lgb_eval  = lgb.Dataset(data=val_x, label=val_y)

params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 5000, 'verbose': 0 ,
          'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
          'min_split_gain':.01, 'min_child_weight':1}

start = time.time()
model = lgb.train(params, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=150, verbose_eval=200)
print("Training took {} seconds".format(round(time.time() - start)))

Training until validation scores don't improve for 150 rounds.
[200]	valid_0's auc: 0.739047
[400]	valid_0's auc: 0.753712
[600]	valid_0's auc: 0.764681
[800]	valid_0's auc: 0.770134
[1000]	valid_0's auc: 0.772733
[1200]	valid_0's auc: 0.774141
[1400]	valid_0's auc: 0.775041
[1600]	valid_0's auc: 0.775742
[1800]	valid_0's auc: 0.776154
[2000]	valid_0's auc: 0.776323
[2200]	valid_0's auc: 0.776499
[2400]	valid_0's auc: 0.77658
Early stopping, best iteration is:
[2367]	valid_0's auc: 0.776621
Training took 544 seconds


### Predictions and save to CSV

In [77]:
predictions = model.predict(test_x)

pd.DataFrame({
    "SK_ID_CURR": test_id,
    "TARGET": predictions
}).to_csv("../submissions/remove_skew.csv", index=False)